# Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов.

# Урок 3. Парсинг HTML. BS, SQLAlchemy

1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД  
2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы  
3)*Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [1]:
import requests as req
from pymongo import MongoClient
from bs4 import BeautifulSoup as bs
from pprint import pprint

In [2]:
db_client = MongoClient('localhost',27017)
db = db_client['hh_sj_scraping']
mongo_hh = db.hh
mongo_sj = db.sj

In [3]:
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) \
Chrome/78.0.3904.97 Safari/537.36'
headers = {'User-Agent': user_agent}

#### Настройки для парсинга - ссылки, теги, атрибуты. чтобы все в одном месте 

In [4]:
hh_params = {'main_link' : 'https://hh.ru',
            'search_link' : '/search/vacancy?text=',
            'links_prefix' : '',
            'links_tags' : ['a', 'div', 'script'],
            'links_attrs' : [{'class':'bloko-link HH-LinkModifier'}, 
                             {'class':'resume-search-item__name'},
                             {'data-name':'HH/AjaxContentLoader'}],
            'vac_tot_tag' : 'a',
            'vac_tot_attr' : {'class':'bloko-link HH-LinkModifier'},
            'pages_tot_tag' : 'a',
            'pages_tot_attr' : {'class':'bloko-button HH-Pager-Control'},
            'vac_name_tag' : 'h1',
            'vac_name_attrs' : {'data-qa':'vacancy-title'},
            'vac_empl_tag' : 'span',
            'vac_empl_attrs' : {'itemprop':'name'},
            'vac_loc_tag' : 'span',
            'vac_loc_attrs' : [{'data-qa':'vacancy-view-raw-address'}, {'itemprop':'jobLocation'}],
            'vac_salary_tag' : 'meta',
            'vac_salary_attrs' : [{'itemprop':'currency'}, {'itemprop':'minValue'}, {'itemprop':'maxValue'}]}

In [5]:
sj_params = {'main_link' : 'https://www.superjob.ru',
            'search_link' : '/vacancy/search/?geo%5Bc%5D%5B0%5D=1&keywords=',
            'links_prefix' : 'https://www.superjob.ru',
            'links_tags' : ['a', 'div'],
            'links_attrs' : [{'target':'_blank'}, {'class':'_3syPg _3P0J7 _9_FPy'}],
            'vac_tot_tag' : 'div',
            'vac_tot_attr' : {'class':'_1tH7S _1o0Xp GPKTZ _3achh _3ofxL _2_FIo'},
            'pages_tot_tag' : 'span',
            'pages_tot_attr' : {'class':'qTHqo _2h9me DYJ1Y _2FQ5q _2GT-y'},
            'vac_name_tag' : 'h1',
            'vac_name_attrs' : {'class':'_3mfro rFbjy s1nFK _2JVkc'},
            'vac_empl_tag' : 'h2',
            'vac_empl_attrs' : {'class':'_3mfro PlM3e _2JVkc _2VHxz _3LJqf _15msI'},
            'vac_loc_tag' : 'span',
            'vac_loc_attrs' : [{'class':'_6-z9f'}],
            'vac_salary_tag' : 'span',
            'vac_salary_attrs' : {'class':'_3mfro _2Wp8I ZON4b PlM3e _2JVkc'}}

**get_vacancy_salary** - собираем зп  
**get_vacancy_data** - собираем остальную информацию  
**get_vacanies** - информация о количестве вакансий и страниц  
**vacancies_scraping** - собираем ссылки вакансий для дальнейшего обхода  
**err_log** - лог со статус-кодами неудачных запросов  
**if not(self.mongo_collection.count_documents({'link':{'$eq': vacancy['link']}}))** - проверяем существование документа с такой ссылкой в базе. если есть, то данную вакансию не добавляем

In [6]:
class Scraper:
    
    def __init__(self, main_link, search_link, mongo_col):
        self.headers = headers
        self.main_link = main_link
        self.search_link = search_link
        self.start_link = ''
        self.vacancies_total = 0
        self.pages_total = 0
        self.pages = 0
        self.err_log = {}
        self.mongo_collection = mongo_col
        self.docs_before = self.mongo_collection.estimated_document_count()
        self.dos_after = self.docs_before
    
    def get_vacancy_salary(self, parsed_html, params):
        if self.main_link == hh_params['main_link']:
            try:
                currency = parsed_html.find(params['vac_salary_tag'], params['vac_salary_attrs'][0])['content']
            except:
                currency = None
            try:
                min_salary = int(parsed_html.find(params['vac_salary_tag'], params['vac_salary_attrs'][1])['content'])
            except:
                min_salary = None
            try:
                max_salary = int(parsed_html.find(params['vac_salary_tag'], params['vac_salary_attrs'][2])['content'])
            except:
                max_salary = None
        else:
            try:
                vac_salary = parsed_html.find(params['vac_salary_tag'], params['vac_salary_attrs']).getText()
            except:
                vac_salary = None
            if vac_salary != None:
                if 'оговорённо' in vac_salary:
                    min_salary = None
                    max_salary = None
                    currency = None
                elif '—' in vac_salary:
                    vac_salary = vac_salary.split('—')
                    currency = vac_salary[1][-1]
                    max_salary = int(''.join(i for i in vac_salary[1][:-1] if i.isdigit()))
                    min_salary = int(''.join(i for i in vac_salary[0] if i.isdigit()))
                elif 'от' in vac_salary:
                    vac_salary = vac_salary.split()
                    currency = vac_salary[-1]
                    max_salary = None
                    min_salary = int(''.join(i for i in vac_salary[1:-1] if i.isdigit()))
                elif 'до' in vac_salary:
                    vac_salary = vac_salary.split()
                    currency = vac_salary[-1]
                    max_salary = int(''.join(i for i in vac_salary[1:-1] if i.isdigit()))
                    min_salary = None
                else:
                    vac_salary = vac_salary.split()
                    currency = vac_salary[-1]
                    max_salary = int(''.join(i for i in vac_salary[:-1] if i.isdigit()))
                    min_salary = int(''.join(i for i in vac_salary[:-1] if i.isdigit()))
            else:
                min_salary, max_salary, currency = [None] * 3
        return currency, min_salary, max_salary
    
    def get_vacancy_data(self, vacancies_links, params):
        for curr_link in vacancies_links:
            response = req.get(curr_link, headers=self.headers)
            if response.status_code == 200:
                parsed_html = bs(response.text,'lxml')  
                vacancy = {}
                try:
                    vacancy['link'] = params['main_link'] + curr_link.split('.hh.ru', 1)[1]
                except:
                    vacancy['link'] = curr_link    
                if not(self.mongo_collection.count_documents({'link':{'$eq': vacancy['link']}})):
                    try:
                        vacancy['name'] = parsed_html.find(params['vac_name_tag'], params['vac_name_attrs']).getText()
                    except:
                        vacancy['name'] = None
                    try:
                        vacancy['employer'] = parsed_html.find(params['vac_empl_tag'], params['vac_empl_attrs']).getText()
                    except:
                        vacancy['employer'] = None
                    try:
                        vacancy['location'] = parsed_html.find(params['vac_loc_tag'], params['vac_loc_attrs'][0]).getText()       
                    except:
                        try:
                            vacancy['location'] = parsed_html.find(params['vac_loc_tag'], params['vac_loc_attrs'][1]).findParent().getText()
                        except:
                            vacancy['location'] = None
                    vacancy['currency'], vacancy['min_salary'], vacancy['max_salary'] = self.get_vacancy_salary(parsed_html, params)
                    vacancy['site_name'] = self.main_link
                    self.mongo_collection.insert_one(vacancy)
            else:
                self.err_log[curr_link] = response.status_code
                
    def get_vacanies(self, params, search_text):
        self.start_link = self.main_link + self.search_link + search_text
        response = req.get(self.start_link, headers=self.headers)
        if response.status_code == 200:
            parsed_html = bs(response.text,'lxml')
            if self.main_link == hh_params['main_link']:
                try:
                    self.vacancies_total = int(parsed_html.find(params[0], params[1])['data-totalvacancies'])
                except:
                    self.vacancies_total = 0
                try:
                    self.pages_total = int(parsed_html.find_all(params[2], params[3])[-1]['data-page']) + 1
                except:
                    if self.vacancies_total > 0:
                        self.pages_total = 1
                    else:
                        self.pages_total = 0
            else:
                try:
                    self.vacancies_total = int(parsed_html.find(params[0], params[1]).findChild().getText().split()[1])                         
                except:
                    self.vacancies_total = 0
                try:
                    self.pages_total = int(parsed_html.find_all(params[2], params[3])[-2].getText())
                except:
                    if self.vacancies_total > 0:
                        self.pages_total = 1
                    else:
                        self.pages_total = 0
            return self.vacancies_total, self.pages_total
        else:
            return print('что-то пошло не так: response.status_code', response.status_code)
        
    def vacancies_scraping(self, params):
        self.err_log = {}
        vacancies_links = []
        for page in range(self.pages):
            if self.main_link == hh_params['main_link']:
                curr_link = self.start_link + f'&page={page}'
            else:
                curr_link = self.start_link + f'&page={page + 1}'
            response = req.get(curr_link, headers=self.headers)
            if response.status_code == 200:
                parsed_html = bs(response.text,'lxml')
                try:
                    vacancies_links = [params['links_prefix'] + item.findChild(params['links_tags'][0], params['links_attrs'][0])['href']\
                                       for item in parsed_html.find_all(params['links_tags'][1], params['links_attrs'][1])]
                except:
                    vacancies_links = []              
                if vacancies_links != []:
                    self.get_vacancy_data(vacancies_links, params)
            else:
                self.err_log[curr_link] = response.status_code
        self.docs_after = self.mongo_collection.estimated_document_count()
        print(f'в коллекцию было добавлено: {self.docs_after - self.docs_before} док.')
        print(f'на данный момент в коллекции: {self.docs_after} док.')
        if self.err_log != {}:
            print('не все прошло гладко - "link":"response.status_code"')
            pprint(self.err_log)

#### фунция выбора вакансий с заданной зп из базы

In [7]:
def mongo_get_salary(collections, salary):
    cols = []
    for col in collections:
        cols.append(col.find({'$or':[{'min_salary':{'$gt':salary}}, {'max_salary':{'$gt':salary}}]}))
    return cols

#### инициализация

In [8]:
hh_scraper = Scraper(hh_params['main_link'], hh_params['search_link'], mongo_hh)
sj_scraper = Scraper(sj_params['main_link'], sj_params['search_link'], mongo_sj)

#### фомируем строку для поиска

In [9]:
search_text = input('Что ищем? ')
if search_text == '':
    search_text = 'Python'

Что ищем? 


In [10]:
hh_scraper.get_vacanies([hh_params['vac_tot_tag'], hh_params['vac_tot_attr'],
                         hh_params['pages_tot_tag'], hh_params['pages_tot_attr']], search_text)
sj_scraper.get_vacanies([sj_params['vac_tot_tag'], sj_params['vac_tot_attr'],
                         sj_params['pages_tot_tag'], sj_params['pages_tot_attr']], search_text)

(95, 5)

#### задаем количество страниц для обхода на hh.ru

In [11]:
print(f'на {hh_params["main_link"]} найдено {hh_scraper.vacancies_total} вакансий по запросу {search_text}')
print(f'всего страниц {hh_scraper.pages_total}')
if hh_scraper.vacancies_total > 0:
    try:
        pages = abs(int(input('Сколько страниц будем обходить? (по умолчанию 0) ')))
        if pages > hh_scraper.pages_total:
            hh_scraper.pages = hh_scraper.pages_total
        else:
            hh_scraper.pages = pages
    except:
        hh_scraper.pages = 0

на https://hh.ru найдено 6521 вакансий по запросу Python
всего страниц 100
Сколько страниц будем обходить? (по умолчанию 0) 1


#### задаем количество страниц для обхода на superjob.ru

In [12]:
print(f'на {sj_params["main_link"]} найдено {sj_scraper.vacancies_total} вакансий по запросу {search_text}')
print(f'всего страниц {sj_scraper.pages_total}')
if sj_scraper.vacancies_total > 0:
    try:
        pages = abs(int(input('Сколько страниц будем обходить? (по умолчанию 0) ')))
        if pages > sj_scraper.pages_total:
            sj_scraper.pages = sj_scraper.pages_total
        else:
            sj_scraper.pages = pages
    except:
        sj_scraper.pages = 0

на https://www.superjob.ru найдено 95 вакансий по запросу Python
всего страниц 5
Сколько страниц будем обходить? (по умолчанию 0) 1


In [13]:
hh_scraper.vacancies_scraping(hh_params)

в коллекцию было добавлено: 0 док.
на данный момент в коллекции: 21 док.


In [14]:
sj_scraper.vacancies_scraping(sj_params)

в коллекцию было добавлено: 0 док.
на данный момент в коллекции: 60 док.


In [15]:
try:
    input_salary = abs(int(input('желаемая зарплата (по умолчанию 100000) ')))
except:
    input_salary = 100000

желаемая зарплата (по умолчанию 100000) 


In [16]:
for i in mongo_get_salary([mongo_hh, mongo_sj], input_salary):
    for item in i:
        print(f"vacancy: {item['name']} | min_salary: {item['min_salary']} | max_salary: {item['max_salary']} | link: {item['link']}")

vacancy: Python разработчик | min_salary: 400000 | max_salary: 600000 | link: https://hh.ru/vacancy/34664011?query=Python
vacancy: Web-программист Python (Middle/Senior) | min_salary: 150000 | max_salary: None | link: https://hh.ru/vacancy/34510621?query=Python
vacancy: Kubernetes Engineer | min_salary: 200000 | max_salary: None | link: https://hh.ru/vacancy/34620646?query=Python
vacancy: QA Automation Engineer | min_salary: 120000 | max_salary: 150000 | link: https://hh.ru/vacancy/32370952?query=Python
vacancy: Ведущий разработчик Python / Software development / Team lead | min_salary: 200000 | max_salary: None | link: https://hh.ru/vacancy/34200113?query=Python
vacancy: Инженер по тестированию / QA Engineer | min_salary: 80000 | max_salary: 110000 | link: https://hh.ru/vacancy/34620026?query=Python
vacancy: QA Automation Engineer | min_salary: 120000 | max_salary: 150000 | link: https://hh.ru/vacancy/32370885?query=Python
vacancy: QA Automation Engineer | min_salary: 120000 | max_sal